## Title: CI Biodiversity Hotspots (version 2016.1)

### Description
There are currently 36 recognized biodiversity hotspots. These are Earth’s most biologically rich—yet threatened—terrestrial regions. To qualify as a biodiversity hotspot, an area must meet two strict criteria: <br>

- Contain at least 1,500 species of vascular plants found nowhere else on Earth (known as "endemic" species).
- Have lost at least 70 percent of its primary native vegetation. <br>

Many of the biodiversity hotspots exceed the two criteria. For example, both the Sundaland Hotspot in Southeast Asia and the Tropical Andes Hotspot in South America have about 15,000 endemic plant species. The loss of vegetation in some hotspots has reached a startling 95 percent.

### FLINT
This dataset has been pre-processed/checked and is suitable for use in FLINT. Please adhere to individual dataset licence conditions and citations. Processed data can be accessed here: https://datasets.mojaglobal.workers.dev/

### Format
<b>Extent: </b>Global coverage<br>
<b>Format</b>: polygon geoJSON .json<br>
<b>Cordinate system:</b> EPSG:4326 (WGS84)<br>
<b>Temporal Resolution: </b>2016 <br>
<b>Size:</b> 18MB 

### Original source
Original Source: https://zenodo.org/record/3261807#.X-p6mtgzZPa Accessed 29/12/2020 <br>

### Licence
This dataset is available under the Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0): https://creativecommons.org/licenses/by-sa/4.0/

### Citation
Michael Hoffman, Kellee Koenig, Gill Bunting, Jennifer Costanza, & Williams, Kristen J. (2016). Biodiversity Hotspots (version 2016.1) (Version 2016.1) [Data set]. Zenodo. http://doi.org/10.5281/zenodo.3261807

### Original format
Global coverage, vector, shapefile<br>
Cordinate system EPSG: 4326 (WGS84)

### Metadata
Version 2016.1. 25 April 2016. Added North American Coastal Plains hotspot (Noss, R.F., Platt, W.J., Sorrie, B.A., Weakley, A.S., Means, D.B., Costanza, J., and Peet, R.K. (2015). How global biodiversity hotspots may go unrecognized: lessons from the North American Coastal Plain. Diversity and Distributions, 21, 236–244.) Hotspot boundary modified to remove overlap with Mesoamerica and Madrean Pine-Oak Woodlands hotspots.

Version 2016. 4 April 2016. Version 2011 with updated Eastern Afromontane hotspot boundary based on improved elevation data (Eastern Afromontane Outcomes profile, BirdLife International, 2016).

Version 2011. Added Forests of Eastern Australia hotspot (Full set of 35 hotspots: Mittermeier, R. A., Turner, W. R., Larsen, F. W., Brooks, T. M., & Gascon, C. (2011). Global biodiversity conservation: The critical role of hotspots. In F. E. Zachos & J. C. Habel (Eds.), Biodiversity Hotspots (pp. 3–22). Berlin Heidelberg: Springer. New hotspot: Williams, K. J., Ford, A., Rosauer, D. F., Silva, N., Mittermeier, R. A., Bruce, C., … Margules, C. (2011). Forests of East Australia: The 35th biodiversity hotspot. In F. E. Zachos & J. C. Habel (Eds.), Biodiversity Hotspots (pp. 295–310). Berlin Heidelberg: Springer.).

Version 2004. Hotspots Revisited (Mittermeier, R. A., Robles Gil, P., Hoffmann, M., Pilgrim, J., Brooks, T., Mittermeier, C. G., … da Fonseca, G. A. B. (2004). Hotspots Revisited: Earth’s Biologically Richest and Most Endangered Ecoregions (p. 390). Mexico City, Mexico: CEMEX.)

Version 2000. Hotspots (Myers, N., Mittermeier, R. A., Mittermeier, C. G., da Fonseca, G. A. B., & Kent, J. (2000). Biodiversity hotspots for conservation priorities. Nature, 403, 853–858.)

### Notes
Has significant overlapping edges and gaps caused by slight topological error over North American coastal plain, Mesoamerica and Polynesia-Micronesia. Code below finds and fixes these by merging into coincident polygons using the longest edge. Some coastlines are slightly offset from actual coastline (+/- 500m).

Processing time is lengthy. Please note, only gaps smaller than 0.5ha will be fixed as gaps larger than this could be valid. Gaps smaller than 0.5ha could also be valid but are removed in this instance for improved drawing speed.

### Processing
Repair geometry, fix topologial error (remove overlaps), convert to geojson, EPSG:4326 (WGS84), remove/disable Z values. View code below - originally processed in ArcGIS but can be converted to open source QGIS or GDAL (or others).

In [ ]:
# Import arcpy module
import arcpy
import os

# Input variables
in_folder = r"C:\Users\LennyJenny\Documents\ArcGIS\world\UNFCCC\downloads\CIBiodiversityHotspots"
scr_folder = r"C:\Data\scratch.gdb"
out_folder = r"C:\Data\json"
field = "NAME IS NULL OR NAME = ''"

# Environments
workspace = in_folder
arcpy.env.workspace = workspace
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)
arcpy.env.outputZFlag = "Disabled"
arcpy.env.overwriteOutput = True
scr = arcpy.CreateFileGDB_management(r"C:\Data", "scratch")
arcpy.env.parallelProcessingFactor = "100%"

# List features to process
featureclasses = arcpy.ListFeatureClasses()
print(featureclasses)

# Repair/check topology and make FLINT ready
for fc in featureclasses:
    fcname = os.path.join(os.path.splitext(fc)[0])
    outjson = os.path.join(out_folder, fcname)
    whereclause = "FID_" +fcname + " =-1 AND AREA_GEO <= 5000 Or AREA_GEO IS NULL"
    print(fcname + ' processing...')
    
    fLayer = "project_Layer"
    arcpy.management.MakeFeatureLayer(fc, fLayer)
    projectIntersect = os.path.join(scr_folder, "projectIntersect")
    arcpy.analysis.Intersect(fLayer, projectIntersect, "ONLY_FID")
    
    projectSingle = os.path.join(scr_folder, "projectSingle")
    arcpy.management.MultipartToSinglepart(projectIntersect, projectSingle)

    dissolveSlither = os.path.join(scr_folder, "dissolveSlither")
    arcpy.management.Dissolve(projectSingle, dissolveSlither, None, None,"SINGLE_PART")
    
    # Take action if overlaps
    if arcpy.management.GetCount(dissolveSlither)[0] == "0":
        print('no overlaps detected...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(fLayer,projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes(projectUnion, "AREA_GEODESIC", None, "SQUARE_METERS")

        uniSelect = os.path.join(scr_folder, "uniSelect")
        arcpy.analysis.Select(projectUnion, uniSelect, whereclause)
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
                     
            # Progress report no error
            print(fcname, 'No gaps and overlaps. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fLayer, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fLayer, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            print('gaps detected')

            appendGap = arcpy.management.Append(uniSelect, fLayer, "NO_TEST")     
            selectGap = arcpy.management.SelectLayerByAttribute(fLayer, "NEW_SELECTION", field)

            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")

            # Progress report 
            print(fcname, 'No overlaps but gaps detected and repaired. Repairing geometry and conversion to json...')

            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")


    else:
        print('Overlaps detected...')
        # Fix overlaps
        projectErase = os.path.join(scr_folder, "projectErase")
        arcpy.analysis.Erase(fLayer, dissolveSlither, projectErase)
        arcpy.management.Append(dissolveSlither, projectErase, "NO_TEST")
    
        selectSlither = arcpy.management.SelectLayerByAttribute(projectErase, "NEW_SELECTION",  field)
        
        eliminateSlither = os.path.join(scr_folder, "eliminateSlither")
        arcpy.management.Eliminate(selectSlither, eliminateSlither, "LENGTH")
        
        print('Overlaps detected and fixed...checking for gaps...')
        
        projectUnion = os.path.join(scr_folder, "projectUnion")
        arcpy.analysis.Union(eliminateSlither, projectUnion, "ALL", None, "NO_GAPS")
        arcpy.management.AddGeometryAttributes(projectUnion, "AREA_GEODESIC", None, "SQUARE_METERS")
        
        uniSelect = os.path.join(scr_folder, "uniSelect")
        
        arcpy.analysis.Select(projectUnion, uniSelect, "FID_eliminateSlither = -1 AND AREA_GEO <=5000 OR AREA_GEO IS NULL")
        
        if arcpy.management.GetCount(uniSelect)[0] == "0":
            
            # Progress report no error
            print(fcname, ' No gaps detected. Repairing geometry and conversion to json...')
    
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(eliminateSlither, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(eliminateSlither, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")

            print(outjson, '.geojson complete')
            
        else:
            # Take action if gaps
            appendGap = arcpy.management.Append(uniSelect, eliminateSlither, "NO_TEST")
            
            selectGap = arcpy.management.SelectLayerByAttribute(eliminateSlither, "NEW_SELECTION",  field)
            
            fixedlyr = os.path.join(scr_folder, "fixedlyr")
            
            arcpy.management.Eliminate(selectGap, fixedlyr, "LENGTH")
            
            print('gaps detected and repaired')
            
            # Progress report 
            print(fcname, 'Gaps and overlaps fixed. Repairing geometry and conversion to json...')
            
            # Process: Repair Geometry (non-simple geometry)
            geomRepair = arcpy.management.RepairGeometry(fixedlyr, "DELETE_NULL", "OGC")[0]

            # Process: Features To JSON
            arcpy.conversion.FeaturesToJSON(fixedlyr, outjson, "NOT_FORMATTED", "NO_Z_VALUES", "NO_M_VALUES", "GEOJSON", "WGS84", "USE_FIELD_NAME")
            
arcpy.AddMessage("All done!")
print('done')